In [1]:
import requests 
import time
import json
from bs4 import  BeautifulSoup
import pandas as pd

In [2]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'
    }

In [3]:
def get_proxy():
    return requests.get("http://zltiqu.pyhttp.taolop.com/getip?count=1&neek=81346&type=2&yys=0&port=1&sb=&mr=1&sep=0&ts=1").json()

def delete_proxy(proxy):
    requests.get("http://127.0.0.1:5010/delete/?proxy={}".format(proxy))

def getHtml(url):
    retry_count = 10
    proxy = get_proxy().get("proxy")
    while retry_count > 0:
        try:
            # 使用代理访问
            html = requests.get(url, proxies={"http": "http://{}".format(proxy)}, headers=headers)
            return html
        except Exception:
            retry_count -= 1
            # 删除代理池中代理
            delete_proxy(proxy)
    print('无可用代理')
    return None

In [4]:
with open('place_info_contain_location.txt', 'r') as f:   
    lines = f.readlines() 
lines_str = str(lines).replace('\'', '"')[2:-2]
city_url_info = json.loads(lines_str)

In [5]:
'''
    获取区域的所有景点
'''
def get_place_url(url):
    response = getHtml(url)
    location_place_url_soup=BeautifulSoup(response.text,'lxml')
    location_place_url_data = location_place_url_soup.select('#tiles > li')

    place_info_list = []

    for item in location_place_url_data:

        try:
            place_info={}
            place_info['href'] = item.a.get('href')

            item2 = item.select('li > div.bar')[0]
            place_info['name'] = item2.attrs['m']
            place_info['ever'] = item2.select('a.ever > span')[0].get_text()
            place_info['want'] = item2.select('a.never > span')[0].get_text()

            place_info_list.append(place_info)

        except Exception as e:
            print(e)
    time.sleep(2)
    return place_info_list

In [7]:
'''
    获取景点信息
'''
def get_place_info(url):

    place_response = getHtml(url)
    place_soup=BeautifulSoup(place_response.text,'html5lib')
    place_header_data = place_soup.select('#jdleft > div.info > div > div.hd > ul > li')
    place_data = place_soup.select('#jdleft > div.info > div > div > div.item')

    place_info_item = {}
    for i in range(len(place_header_data)):

        if place_header_data[i].get_text() in ['学校介绍', '景区介绍', '旅游攻略', '怎么去', '门票']:
            # print(place_header_data[i].get_text())
            # print(place_data[i])
            item_header = place_data[i].find_all('h2')
            # print(item_header)
            place_data_text = place_data[i].get_text()
            
            if place_header_data[i].get_text()=='旅游攻略':
                lv_gl_text_data = place_data[i].select('ul')
                if len(lv_gl_text_data) > 0:
                    lv_gl_text = lv_gl_text_data[0].get_text()
                    place_data_text = place_data_text.replace(lv_gl_text, '')
            
            place_data_text = place_data_text.replace('信息纠错', '')
            place_data_text = place_data_text.replace('景区的卫星地图', '')

            

            place_info_item[place_header_data[i].get_text()] = {}

            for j in range(len(item_header)):
                current_header = item_header[j].get_text()[:-1]
                current_text = place_data_text[place_data_text.find(current_header)+len(current_header)+1:]

                if j < len(item_header)-1:
                    next_header = item_header[j+1].get_text()
                    current_text = current_text[:current_text.find(next_header)]
                
                if current_header not in ['景区游览图', '学校地图', '景区地图', '驴友游记']:
                    place_info_item[place_header_data[i].get_text()][current_header] = current_text.replace('　　', '').replace(' ', '').split('\n')
    time.sleep(2)
    
    return place_info_item



In [8]:
location_error_list = []
error_list = []
base_url = 'https://www.meet99.com'
rlt = {'name':[], 'province': [], 'city': [], 'location': [], 'desc': [], 'want': [], 'has_been': [], 'strategy': [], 'route': [], 'ticket_info': [], 'open_time': []}

for province_key in city_url_info.keys():
    if province_key in ['辽宁省']:
        for city_key in city_url_info[province_key]['children'].keys():
            # print(city_key)
            # if city_key in ['沈阳市']:
            #     continue
            
            if 'children' in city_url_info[province_key]['children'][city_key] and len(city_url_info[province_key]['children'][city_key]['children'].keys()) > 0 :
                
                for location_key in city_url_info[province_key]['children'][city_key]['children'].keys():
                    # if city_key =='大连市' and location_key in ['中山区', '西岗区', '沙河口区', '甘井子区']:
                    #     continue
                    location_info = city_url_info[province_key]['children'][city_key]['children'][location_key]
                    location_url = base_url + location_info['link']
                    print(location_key)
                    print(location_url)
                    location_place_info_list = []
                    try:
                        location_place_info_list = get_place_url(location_url)

                    except Exception as e:
                        print('获取区域内的景点出现异常,异常原因:', e)
                        print(location_info)
                        location_error_info = {}
                        location_error_info['province_key'] = province_key
                        location_error_info['city_key'] = city_key
                        location_error_info['location_name'] = location_key
                        location_error_info['location_info'] = location_info
                        location_error_list.append(location_error_info)


                    for place_item in location_place_info_list:
                        try:
                            print(place_item)
                            place_url = base_url + place_item['href']
                            place_info = get_place_info(place_url)

                            name = place_item['name']
                            
                            province = province_key
                            city = city_key
                            location = location_key
                            desc = {}
                            if '景区介绍' in place_info.keys():
                                desc = place_info['景区介绍']
                            elif '学校介绍' in place_info.keys():
                                desc = place_info['学校介绍']

                            want = place_item['want']
                            has_been = place_item['ever']

                            strategy = {}
                            if '旅游攻略' in place_info.keys():
                                strategy = place_info['旅游攻略']

                            route = {}
                            if '怎么去' in place_info.keys():
                                route = place_info['怎么去']

                            ticket_info = {}
                            open_time = {}
                            if '门票' in place_info.keys():
                                for key in place_info['门票']:
                                    if key not in ['景区开放时间']:
                                        ticket_info[key] = place_info['门票'][key]


                                if '景区开放时间' in place_info['门票'].keys():
                                    open_time = place_info['门票']['景区开放时间']
                            
                            ticket = ticket_info

                            rlt['name'].append(name)
                            rlt['province'].append(province)
                            rlt['city'].append(city)
                            rlt['location'].append(location)
                            rlt['desc'].append(desc)
                            rlt['want'].append(want)
                            rlt['has_been'].append(has_been)
                            rlt['strategy'].append(strategy)
                            rlt['route'].append(route)
                            rlt['ticket_info'].append(ticket_info)
                            rlt['open_time'].append(open_time)
                            

                        except Exception as e:
                            print('以下景点出现异常,异常原因:', e)
                            print(place_item)
                            error_info = {}
                            error_info['province_key'] = province_key
                            error_info['city_key'] = city_key
                            error_info['location_name'] = location_key
                            error_info['place_info'] = place_item
                            error_list.append(error_info)  

                        time.sleep(2)

                    time.sleep(2)

                time.sleep(2)

                    


旅顺口区
https://www.meet99.com/lvyou-lvshunkou.html


'NoneType' object has no attribute 'get'
{'href': '/jingdian-lvshukoufengjingmingshengqu.html', 'name': '大连海滨旅顺口', 'ever': '5223', 'want': '1063'}
{'href': '/jingdian-lvshunkoubaiyushan.html', 'name': '旅顺口白玉山景区', 'ever': '2369', 'want': '363'}
{'href': '/jingdian-lvshunrierjianyu.html', 'name': '旅顺日俄监狱旧址博物馆', 'ever': '1120', 'want': '226'}
{'href': '/jingdian-dalianshijiehepinggongyuan.html', 'name': '大连世界和平公园', 'ever': '688', 'want': '261'}
{'href': '/jingdian-lvshunkoudongjiguanshan.html', 'name': '旅顺口东鸡冠山', 'ever': '616', 'want': '135'}
{'href': '/jingdian-lslts.html', 'name': '大连旅顺老铁山', 'ever': '490', 'want': '85'}
{'href': '/jingdian-lvshun203gaodi.html', 'name': '旅顺203高地新樱花园景区', 'ever': '188', 'want': '46'}
{'href': '/jingdian-dllsbwg.html', 'name': '大连旅顺博物馆', 'ever': '345', 'want': '75'}
{'href': '/jingdian-beiputuo.html', 'name': '大连横山北普陀主题文化公园', 'ever': '47', 'want': '38'}
{'href': '/jingdian-lvshuntianmuwenquan.html', 'name': '旅顺天沐温泉', 'ever': '61', 'want': '22'}
{'href': '/j

In [9]:
rlt_df = pd.DataFrame(rlt)

In [14]:
rlt_df

,name,province,city,location,desc,want,has_been,strategy,route,ticket_info,open_time
0,仙女湖公园,辽宁省,沈阳市,铁西区,{'景区简介': ['仙女湖公园位于沈阳铁西新区，公园占地面积8.6万平方米，其中水面面积4...,29,158,{'最佳游览时间': ['春夏秋']},{'景区位置': ['中国>辽宁省>沈阳市>铁西区>大青中朝友谊街道>翟家街道>大潘街道>彰...,{'仙女湖公园门票价格': ['无']},[全天]
1,沈阳中国工业博物馆,辽宁省,沈阳市,铁西区,{'景区简介': ['中国工业博物馆位于沈阳市铁西区，于2012年5月18日世界博物馆日开馆...,17,77,{'最佳游览时间': ['四季皆宜']},{'景区位置': ['中国>辽宁省>沈阳市>铁西区>大青中朝友谊街道>翟家街道>大潘街道>彰...,{'中国工业博物馆门票价格': ['免费']},"[每周二至周日9:00—16:00。, 每周一休息，逢法定重大节假日休馆。]"
2,沈阳奉达公园,辽宁省,沈阳市,铁西区,{'景区简介': ['沈阳奉达公园位于铁西区昆明湖街，是附近居民休闲散步的好地方。公园内广植...,12,43,{'最佳游览时间': ['四季皆宜']},{'景区位置': ['中国>辽宁省>沈阳市>铁西区>大青中朝友谊街道>翟家街道>大潘街道>彰...,{'奉达公园门票价格': ['免费']},[全天]
3,沈阳水洞风景区,辽宁省,沈阳市,苏家屯区,{'景区简介': ['沈阳水洞置身于群山怀抱之中，洞外山川秀丽，鸟语花香，集自然与人文景观于...,545,344,"{'最佳游览时间': ['夏季'], '购物推荐': ['蕨菜、婆婆丁、山辣椒等，苹果梨、葡...","{'景区位置': ['中国>辽宁省>沈阳市>苏家屯区>沈水街道'], '到达方式': ['公...","{'水洞门票价格': ['旺季：60元/人；', '淡季：40元/人']}",[8:00-18:00（4-10月）；8:00-17:00（11-3月）]
4,沈阳绿岛旅游度假区,辽宁省,沈阳市,苏家屯区,{'景区简介': ['绿岛旅游度假区，是一处集休闲度假、体育健身、农业观光和北方民俗风情战士...,343,329,"{'旅游攻略': ['景区活动：', '抹黑节：农历正月十六日，', '撒路灯:正月十五']...","{'景区位置': ['中国>辽宁省>沈阳市>苏家屯区>沈水街道'], '到达方式': ['度...",{'绿岛门票价格': ['免费']},[全天]
...,...,...,...,...,...,...,...,...,...,...,...
92,旅顺电岩炮台,辽宁省,大连市,旅顺口区,{},13,30,{},{},{},{}
93,大连龙王塘水库,辽宁省,大连市,旅顺口区,{},1,43,{},{},{},{}
94,旅顺万忠墓纪念馆,辽宁省,大连市,旅顺口区,{},1,46,{},{},{},{}
95,大连旅顺口真泉寺,辽宁省,大连市,旅顺口区,{},0,28,{},{},{},{}


In [15]:
rlt_df.to_csv('辽宁景点_v3.csv', header=True, index=False)

In [10]:
rlt_df['location'].drop_duplicates()

0            海拉尔区
5     莫力达瓦达斡尔族自治旗
7          鄂伦春自治旗
11        鄂温克族自治旗
15          陈巴尔虎旗
17         新巴尔虎左旗
21           满洲里市
27           牙克石市
30           扎兰屯市
37          额尔古纳市
42            根河市
46          扎赉诺尔区
47            临河区
49            五原县
Name: location, dtype: object